<a href="https://colab.research.google.com/github/trybalad/BERT/blob/master/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spacy
!python -m spacy download pl_core_news_lg

[+] Download and installation successful
You can now load the model via spacy.load('pl_core_news_lg')


In [1]:
#Jeśli szkolimy z gpu
!pip install tensorflow-gpu

  Using cached https://files.pythonhosted.org/packages/aa/e7/53bc896aa4e11a87aac10a625c676b3a3d57d1c8d9929e4809d31fa0b7d5/keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b6/48/583c032b79ae5b3daa02225a675aeb673e58d2cb698e78510feceb11958c/gast-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/39/ceabe8fa912cb27ec9a0064fe01c1fde2f8b43e9f7e506207db0a0dcea0a/h5py-3.1.0-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/ec/78/b27f73e923becc6e79e18fe112cf75e3200d1ee35b0dba8fa46181bce56c/tensorflow_estimator-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/48/19/f2090f7dad41e225c7f2326e4cfe6fff49e57dedb5b53636c9551f86b069/cached_property-1.5.2-py2.py3-none-any.whl
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstalling

tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.5.0 which is incompatible.
tensorflow-directml 1.15.4.dev201216 has requirement gast==0.2.2, but you'll have gast 0.4.0 which is incompatible.
tensorflow-directml 1.15.4.dev201216 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
tensorflow-directml 1.15.4.dev201216 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.5.0 which is incompatible.
tensorflow-directml 1.15.4.dev201216 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.5.0 which is incompatible.
Could not install packages due to an E

In [ ]:
#Jeśli wykorzystujemy cpu
!pip install tensorflow

In [1]:
from keras_bert.data_generator import DataGenerator
from keras_bert.model import create_model
from keras_bert.tokenizer import Tokenizer
from keras_bert.training import train_model

In [2]:
#Reading vocab.
tokenizer = Tokenizer()
tokenizer.read_vocab('./data/counted_vocab.txt')
tokenizer.change_to_reversible()
print("Vocab of size:", tokenizer.vocab_size, "loaded.")

Vocab of size: 38999 loaded.


In [ ]:
max_len = 32
embedding_dim = 512
ff_dim = 512
heads = 4
encoder_num = 4

checkpoint_file_path = "./data/checkpoint_notebook.ckpt"
load_checkpoint = False

In [ ]:
#creating data generator
data_generator = DataGenerator("./data/corpus_clean.txt", max_len, tokenizer, batch_size=64, create_nsr_output=True)
print("Data generator prepared.")

In [ ]:
#preparing model
sequence_encoder = create_model(tokenizer.vocab_size, max_len, embedding_dim, encoder_num, heads, encoder_num)
print("Model created.")

In [ ]:
# Start training.
train = train_model(sequence_encoder, max_len, tokenizer, data_generator, epochs=100,
                    checkpoint_file_path=checkpoint_file_path, load_checkpoint=load_checkpoint)

In [ ]:
from keras_bert.prepare_data import create_tokens, create_masks, create_ids, create_segments, translate_ids, \
    create_pretrain_data
import numpy as np

In [ ]:
message = "Pojęcie w programowaniu obiektowym najczyściej utożsamiane z klasą. Jest modelem, który w rzeczywistości nie reprezentuje żadnego istniejącego obiektu, a jedynie na podstawie którego definiowane są inne obiekty."
tokens = create_tokens([message], tokenizer, max_len)
mask_t = create_pretrain_data(tokens, tokenizer)
ids = create_ids(mask_t, max_len, tokenizer)
mask = create_masks(mask_t, max_len)
segments = create_segments(mask_t, max_len)

result = model.predict(x = [np.array(ids), np.array(segments), np.array(mask)])
print(tokens)
print(mask_t)
print(ids)
print(translate_ids(result, tokenizer))